In [2]:
from utils import first_block_of_day, last_block_of_day, exeSQL
import os, pickle

date_start = '2015-10-01'
date_end = '2015-12-31'
block_start = first_block_of_day(int(date_start[:4]), int(date_start[5:7]), int(date_start[8:10]))
block_end = last_block_of_day(int(date_end[:4]), int(date_end[5:7]), int(date_end[8:10]))

action_table_nointernal_name = "action_20151001_20151231_nointernal"
action_table_nointernal_noreward_name = "action_20151001_20151231_nointernal_noreward"
feature_table_name = "cluster_features_20151001_20151231"

In [20]:
# Fetch addrs which solve pow algorithm
def fetch_addrs_set_solving_pow():
    if os.path.isfile('addrs_set_solving_pow_2015_4.pkl'):
        with open('addrs_set_solving_pow_2015_4.pkl', 'rb') as f:
            addrs_solving_pow_set = pickle.load(f)
    else:
        fetch_addrs_solving_pow_sql = """select target from {} where directive='reward-block'""".format(action_table_name)
        addrs_solving_pow_sql_result = exeSQL(fetch_addrs_solving_pow_sql)
        addrs_solving_pow_set = set([r[0] for r in addrs_solving_pow_sql_result])
        with open('addrs_set_solving_pow_2015_4.pkl', 'wb') as f:
            pickle.dump(addrs_solving_pow_set, f)
    return addrs_solving_pow_set

In [24]:
def fetch_total_addr():
    if os.path.isfile('total_addr_set_20151001_20151231.pkl'):
        with open('total_addr_set_20151001_20151231.pkl', 'rb') as f:
            total_addr_set = pickle.load(f)
    else:
        source_set = fetchAddressSet('source')
        target_set = fetchAddressSet('target')
        total_addr_set = source_set.union(target_set)
        with open('total_addr_set_20151001_20151231.pkl', 'wb') as f:
            pickle.dump(total_addr_set, f)
    return total_addr_set

total_addr_set = fetch_total_addr()
# addrs_mark = {}
# for addr in total_addr_set:
#     addrs_mark[addr] = []
print ('0x2a65aca4d5fc5b5c859090a6c34d164135398226' in total_addr_set)

True


In [10]:
# Mark possible mining pools as 1
def mark_possible_mining_pool_condition1():
    mark_num = 1
    addrs_pool_sql = """select address from {} where out_num >= 8 and out_dec_places >= 8 
            and out_ethers >= 0.5 and out_addrs_set_len >= 3""".format(feature_table_name)
    addrs_pool_sql_result = exeSQL(addrs_pool_sql)
    count_dict = {}
    addrs_pool_list = [r[0] for r in addrs_pool_sql_result]
    addrs_pool_list_set = set(addrs_pool_list)
    return addrs_pool_list_set
#     for r in addrs_pool_sql_result:
#         if r[0] not in count_dict:
#             count_dict[r[0]] = 1
#         else:
#             count_dict[r[0]] += 1
#     return count_dict
    
#     for r in addrs_pool_sql_result:
#         addrs_mark[r[0]].append(mark_num)

In [7]:
# Mark possible cooperative miners as 2
def mark_possible_co_miners():
    mark_num = 2
    addrs_co_miners_sql = """select address from {} where in_dec_places >= 8""".format(feature_table_name)
    addrs_co_miners_sql_result = exeSQL(addrs_co_miners_sql)
    for r in addrs_co_miners_sql_result:
        addrs_mark[r[0]].append(mark_num)
    return [r[0] for r in addrs_co_miners_sql_result]
co_miners_2 = mark_possible_co_miners()

In [9]:
print (co_miners_2[:10])

['0x9c56158cb63f026e7f80e2ba0f0d510dd9bbaf4b', '0xe01debe6af51809320e16f9ff1cb0be8dd666a0c', '0xe01debe6af51809320e16f9ff1cb0be8dd666a0c', '0xd6cbc00b3ecff6e48a1106feaf62f77c4f52e465', '0x375bbb5f739f20f8cec43d86b787c8a1ec4576da', '0x5efd26b739a4398bf9b3b9356a5119bcc4ad543a', '0x5efd26b739a4398bf9b3b9356a5119bcc4ad543a', '0x5efd26b739a4398bf9b3b9356a5119bcc4ad543a', '0x5efd26b739a4398bf9b3b9356a5119bcc4ad543a', '0x5efd26b739a4398bf9b3b9356a5119bcc4ad543a']


In [ ]:
# Mark possible cooperative miners as 3
def mark_possible_co_miners():
    mark_num = 3
    addrs_co_miners_sql = """select address from {} where abs((in_ethers - out_ethers)/in_ethers) < 0.1""".format(feature_table_name)
    addrs_co_miners_sql_result = exeSQL(addrs_co_miners_sql)
    for r in addrs_co_miners_sql_result:
        addrs_mark[r[0]].append(mark_num)
    return [r[0] for r in addrs_pool_sql_result]


In [22]:
def fetch_mining_pools():
    # Fetch addrs with massive out transactsions
    def fetch_addrs_massive_out_txs():
        addrs_massive_out_txs_sql = """ select source, ct from (select source, count(target) as ct from
                {} group by source) A where ct > 100""".format(action_table_nointernal_noreward_name)
        addrs_massive_out_txs_sql_result = exeSQL(addrs_massive_out_txs_sql)
        addrs_massive_out_txs_set = set([r[0] for r in addrs_massive_out_txs_sql_result])
        return addrs_massive_out_txs_set
    
    # Fetch addrs with multiple different out addrs
    def fetch_addrs_multiple_outaddrs_txs():
        addrs_multiple_outaddrs_txs_sql = """ select source, dt from (select source, count(distinct(target)) as dt from
                {} group by source) A where dt > 15""".format(action_table_nointernal_noreward_name)
        addrs_multiple_outaddrs_txs_sql_result = exeSQL(addrs_multiple_outaddrs_txs_sql)
        addrs_multiple_outaddrs_txs_set = set([r[0] for r in addrs_multiple_outaddrs_txs_sql_result])
        return addrs_multiple_outaddrs_txs_set
    
    if os.path.isfile('pools_set_2015_4.pkl'):
        with open('pools_set_2015_4.pkl', 'rb') as f:
            pools_set = pickle.load(f)
    else:
        addrs_multiple_outaddrs_txs_set = fetch_addrs_multiple_outaddrs_txs()
        addrs_massive_out_txs_set = fetch_addrs_massive_out_txs()
        addrs_solving_pow_set = fetch_addrs_set_solving_pow()
        pools_set = addrs_massive_out_txs_set.intersection(addrs_solving_pow_set).intersection(addrs_multiple_outaddrs_txs_set)

        # Filter out the addrs with massive tiny out transactions
        pools_set.remove('0xcf684dfb8304729355b58315e8019b1aa2ad1bac')
        
        with open('pools_set_2015_4.pkl', 'wb') as f:
            pickle.dump(pools_set, f)
        
    return pools_set

print (fetch_mining_pools())

{'0x2a65aca4d5fc5b5c859090a6c34d164135398226', '0xf8b483dba2c3b7176a3da549ad41a48bb3121069', '0x1dcb8d1f0fcc8cbc8c2d76528e877f915e299fbe', '0x22a0fbf89ad1362d74f626436d8c4fc6dc4f0679', '0x52bc44d5378309ee2abf1539bf71de1b7d7be3b5', '0xd1e56c2e765180aa0371928fd4d1e41fbcda34d4', '0x68795c4aa09d6f4ed3e5deddf8c2ad3049a601da', '0x4bb96091ee9d802ed039c4d1a5f6216f90f81b01', '0x9d551f41fed6fc27b719777c224dfecce170004d', '0xa027231f42c80ca4125b5cb962a21cd4f812e88f', '0x63a9975ba31b0b9626b34300f7f627147df1f526', '0xa5a5d0bcf280c83eae64e846033fc835afc8961e'}


In [41]:
def fetch_standalone_miners():
    if os.path.isfile('sminers_set_2015_4.pkl'):
        with open('sminers_set_2015_4.pkl', 'rb') as f:
            standalone_miners_set = pickle.load(f)
    else:
        addrs_solving_pow_set = fetch_addrs_set_solving_pow()
        pools_set = fetch_mining_pools()
        standalone_miners_set = addrs_solving_pow_set.difference(pools_set)
        with open('sminers_set_2015_4.pkl', 'wb') as f:
            pickle.dump(standalone_miners_set, f)
    return standalone_miners_set

standalone_miners_set = fetch_standalone_miners()

In [53]:
print (len(standalone_miners_set))

605


In [56]:
def fetch_cooperative_miners():
    if os.path.isfile('cominers_set_2015_4.pkl'):
        with open('cominers_set_2015_4.pkl', 'rb') as f:
            cooperative_miners_set = pickle.load(f)
    else:
        cooperative_miners_set = set()
        pools_set = fetch_mining_pools()
        for pool in pools_set:
            fetch_recipients_from_pools_sql = """select target from {} where source= '{}'
                    """.format(action_table_nointernal_noreward_name, pool)
            fetch_recipients_result = exeSQL(fetch_recipients_from_pools_sql)
            for r in fetch_recipients_result:
                cooperative_miners_set.add(r[0])
        with open('cominers_set_2015_4.pkl', 'wb') as f:
            pickle.dump(cooperative_miners_set, f)
    return cooperative_miners_set

co_miners_set = fetch_cooperative_miners()

In [57]:
print (len(co_miners))

3910


In [30]:
def fetch_exchanges():
    if os.path.isfile('exchanges_set_2015_4.pkl'):
        with open('exchanges_set_2015_4.pkl', 'rb') as f:
            exchanges_set = pickle.load(f)
    else:
    
        # Fetch addrs with massive out transactsions, recipients are not smart contracts, recipients are diverse
        def fetch_addrs_massive_out_txs():
            addrs_massive_out_txs_sql = """ select source, cdt from (select source, count(distinct(target))
                as cdt from {} left join account on 
                {}.target=account.address where 
                account.kind='normal' group by source) A where cdt > 50""".format(action_table_nointernal_noreward_name,
                                                                                 action_table_nointernal_noreward_name)
            addrs_massive_out_txs_sql_result = exeSQL(addrs_massive_out_txs_sql)
            addrs_massive_out_txs_set = set([r[0] for r in addrs_massive_out_txs_sql_result])
            return addrs_massive_out_txs_set
        # Fetch addrs with massive out transactsions，amounts are diverse
        def fetch_addrs_massive_out_amount_txs():
            addrs_massive_amount_txs_sql = """ select source, cda from (select source, count(distinct(amount))
                as cda from {} group by source) A where cda > 50""".format(action_table_nointernal_noreward_name)
            addrs_massive_amount_txs_sql_result = exeSQL(addrs_massive_amount_txs_sql)
            addrs_massive_amount_txs_set = set([r[0] for r in addrs_massive_amount_txs_sql_result])
            return addrs_massive_amount_txs_set

        # Fetch addrs with massive in transactsions, recipients are not smart contracts, recipients are diverse
        def fetch_addrs_massive_in_txs():
            addrs_massive_in_txs_sql = """ select target, cds from (select target, count(distinct(source))
                as cds from {} left join account on 
                {}.source=account.address where 
                account.kind='normal' group by target) A where cds > 50""".format(action_table_nointernal_noreward_name,
                                                                                 action_table_nointernal_noreward_name)
            addrs_massive_in_txs_sql_result = exeSQL(addrs_massive_in_txs_sql)
            addrs_massive_in_txs_set = set([r[0] for r in addrs_massive_in_txs_sql_result])
            return addrs_massive_in_txs_set

        # Fetch addrs with massive in transactsions，amounts are diverse
        def fetch_addrs_massive_in_amount_txs():
            addrs_massive_amount_txs_sql = """ select target, cda from (select target, count(distinct(amount))
                as cda from {} group by target) A where cda > 50""".format(action_table_nointernal_noreward_name)
            addrs_massive_amount_txs_sql_result = exeSQL(addrs_massive_amount_txs_sql)
            addrs_massive_amount_txs_set = set([r[0] for r in addrs_massive_amount_txs_sql_result])
            return addrs_massive_amount_txs_set

        addrs_massive_out_txs_set = fetch_addrs_massive_out_txs()
        addrs_massive_amount_out_txs_set = fetch_addrs_massive_out_amount_txs()
        exchanges_out_set = addrs_massive_out_txs_set.intersection(addrs_massive_amount_out_txs_set)

        addrs_massive_in_txs_set = fetch_addrs_massive_in_txs()
        addrs_massive_amount_in_txs_set = fetch_addrs_massive_in_amount_txs()
        exchanges_in_set = addrs_massive_in_txs_set.intersection(addrs_massive_amount_in_txs_set)

        pools_set = fetch_mining_pools()
        exchanges_set = exchanges_out_set.union(exchanges_in_set).difference(pools_set)
        
        with open('exchanges_set_2015_4.pkl', 'wb') as f:
            pickle.dump(exchanges_set, f)
        
    return exchanges_set
exchanges = fetch_exchanges()

In [31]:
print (len(exchanges))

17


In [3]:
def fetch_addrs_massive_out_txs():
    addrs_massive_out_txs_sql = """ select source, cdt from (select source, count(distinct(target))
        as cdt from {} left join account on 
        {}.target=account.address where 
        account.kind='normal' group by source) A where cdt > 50""".format(action_table_nointernal_noreward_name,
                                                                         action_table_nointernal_noreward_name)
    addrs_massive_out_txs_sql_result = exeSQL(addrs_massive_out_txs_sql)
    addrs_massive_out_txs_set = set([r[0] for r in addrs_massive_out_txs_sql_result])
    return addrs_massive_out_txs_set

addrs_massive_out_txs_set = fetch_addrs_massive_out_txs()

In [9]:
print (len(addrs_massive_out_txs_set))

68


In [8]:
def fetch_addrs_massive_amount_txs():
    addrs_massive_amount_txs_sql = """ select source, cda from (select source, count(distinct(amount))
        as cda from {} group by source) A where cda > 50""".format(action_table_nointernal_noreward_name)
    addrs_massive_amount_txs_sql_result = exeSQL(addrs_massive_amount_txs_sql)
    addrs_massive_amount_txs_set = set([r[0] for r in addrs_massive_amount_txs_sql_result])
    return addrs_massive_amount_txs_set

addrs_massive_amount_txs_set = fetch_addrs_massive_amount_txs()
print (len(addrs_massive_amount_txs_set))

1081


In [18]:
addrs_union_set = addrs_massive_out_txs_set.intersection(addrs_massive_amount_txs_set)
print (len(addrs_union_set))

24
